In [1]:
import numpy as np
import os
import tensorflow as tf

###### Do not modify here ###### 

# to make this notebook's output stable across runs
def reset_graph(seed=42):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)

reset_graph()

from tensorflow.examples.tutorials.mnist import input_data

print("Loading Data...")
# mnist = input_data.read_data_sets("/tmp/data/", one_hot=True)
mnist = input_data.read_data_sets("/tmp/data/")
print(type(mnist.train))

# training on MNIST but only on digits 0 to 4
X_train1 = mnist.train.images[mnist.train.labels < 5]
y_train1 = mnist.train.labels[mnist.train.labels < 5]
#X_valid1 = mnist.validation.images[mnist.validation.labels < 5]
#y_valid1 = mnist.validation.labels[mnist.validation.labels < 5]
X_test1 = mnist.test.images[mnist.test.labels < 5]
y_test1 = mnist.test.labels[mnist.test.labels < 5]

def add_layer(name, inputs, in_size, out_size, activation_function=None):
    # add one more layer and return the output of this layer
    #Weights = tf.Variable(tf.random_normal([in_size, out_size]))
    Weights = tf.get_variable(name, shape=[in_size, out_size], initializer=tf.contrib.layers.variance_scaling_initializer(factor=2.0,
        mode='FAN_IN', uniform=False, seed=None, dtype=tf.float32))
    #biases = tf.Variable(tf.zeros([1, out_size]) + 0.1)
    #biases = tf.get_variable("b"+name, shape=[1, out_size], initializer=tf.contrib.layers.variance_scaling_initializer(factor=2.0,
        #mode='FAN_IN', uniform=False, seed=None, dtype=tf.float32))
    Wx_plus_b = tf.matmul(inputs, Weights) #+ biases
    if activation_function is None:
        outputs = Wx_plus_b
    else:
        outputs = activation_function(Wx_plus_b)
    return outputs

def next_batch(num, data, labels):
    '''
    Return a total of `num` random samples and labels. 
    '''
    idx = np.arange(0 , len(data))
    np.random.shuffle(idx)
    idx = idx[:num]
    data_shuffle = [data[ i] for i in idx]
    labels_shuffle = [labels[ i] for i in idx]

    return np.asarray(data_shuffle), np.asarray(labels_shuffle)

def get_accuracy(predict, label):
    correct_prediction = tf.equal(tf.round(predict), tf.one_hot(label,5))
    #correct_prediction = tf.equal(tf.argmax(predict, 1), label)
    print(sess.run(tf.argmax(predict, 1)[101], feed_dict = {xs: X_test1}))
    return tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

depth = 1
_, dim = X_train1.shape
learning_rate = 0.001
batch_size = 4096

#y_train1 = y_train1[:, np.newaxis]

# define placeholder for inputs to network
xs = tf.placeholder(tf.float32, [None, dim])
ys = tf.placeholder(tf.int64, [None, ])

# add hidden layer
l1 = add_layer("1",xs, dim, 128, activation_function=tf.nn.elu)
l2 = add_layer("2", l1, 128, 128, activation_function=tf.nn.elu)
l3 = add_layer("3", l2, 128, 128, activation_function=tf.nn.elu)
l4 = add_layer("4", l3, 128, 128, activation_function=tf.nn.elu)
l5 = add_layer("5", l4, 128, 128, activation_function=tf.nn.elu)

y = tf.nn.softmax(add_layer("output", l5, 128, 5, activation_function=None))

cross_entropy = tf.reduce_sum(tf.nn.sparse_softmax_cross_entropy_with_logits(labels = ys, logits = y))
train_step = tf.train.AdamOptimizer(learning_rate).minimize(cross_entropy)

sess = tf.InteractiveSession()

tf.global_variables_initializer().run()

for i in range(30):
    
    batch_xs, batch_ys = next_batch(batch_size, X_train1, y_train1)
    sess.run(train_step, feed_dict={xs: batch_xs, ys: batch_ys})
    if(i%5==0):
        print("Steps:",i)
        accuracy = get_accuracy(y,ys)
        print("Accuracy rate:", sess.run(accuracy, feed_dict={xs: X_test1, ys: y_test1}))
    #sess.run(train_step, feed_dict={xs: X_train1, ys: y_train1})

print("Training Finish")

Loading Data...
Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz
<class 'tensorflow.contrib.learn.python.learn.datasets.mnist.DataSet'>
Steps: 0
3
Accuracy rate: 0.840903
Steps: 5
3
Accuracy rate: 0.968827
Steps: 10
2
Accuracy rate: 0.982137
Steps: 15
2
Accuracy rate: 0.984356
Steps: 20
2
Accuracy rate: 0.986885
Steps: 25
3
Accuracy rate: 0.987586
Training Finish


In [2]:
print("Accuracy rate:", sess.run(get_accuracy(y,ys), feed_dict={xs: X_test1, ys: y_test1}))

2
Accuracy rate: 0.98922
